This notebook demonstrates basic usage of the firefly_client API for Firefly Slate Viewer to render tables, images and charts in a grid layout style. This notebook shows the same layout content as what ffapi-api-slate2.html renders in Firefly.  

Note that it may be necessary to wait for some cells (like those displaying an image) to complete before executing later cells.


## Setup

Imports for firefly_client

This example uses the FIREFLY_URL environment variable as firefly server.

'slate.html' is a template made for grid view.

You can also set `start_browser_tab=True` (default is `False`) to open the firefly server in a new browser tab (instead of a lab tab). Although, it only works when running the notebook locally.

In [ ]:
from firefly_client import FireflyClient
import astropy.utils.data

fc = FireflyClient.make_lab_client(start_browser_tab=False)

## Display tables, images, and charts in Window/Grid like layout

Each rendered unit on Firefly Slate Viewer is called a'cell'. To render a cell and its content, the cell location (row, column, width, height), element type and cell ID are needed. (row, column) and (width, height) represent the position and size of the cell in terms of the grid blocks on Firefly Slate Viewer. Element types include types of 'tables', images', 'xyPlots', 'tableImageMeta' or 'coverageImage'. 

Entering a target to search the catalog and image. The coordinates are in decimal hours and decimal degrees for EQ_J2000. The Galactic coordinates are in degrees

In [ ]:
target = '10.68479;41.26906;EQ_J2000' #Galaxy M31
# other notable targets to try
#target = '148.88822;69.06529;EQ_J2000'  #Galaxy M81
#target = '202.48417;47.23056;EQ_J2000'  #Galaxy M51
#target = '136.9316774;+1.1195886;galactic' # W5 star-forming region


## Display tables, images and charts by using plotly


-  All plots are added in one notebook cell to better simulate an application
-  Before re-executing this cell (perhaps with a different target selected), reload the Firefly browser window to clear out the existing plots.

In [ ]:
# Add table in cell 'main'. 
# 'main' is the cell id currently supported by Firefly for element type 'tables'
# this cell is shown at row = 0, col = 4 with width = 2, height = 2

r = fc.add_cell(0, 4, 2, 2, 'tables', 'main')

if r['success']:
    
    fc.show_table(tbl_id='wiseCatTbl', title='WISE catalog', 
                  target_search_info={'catalogProject': 'WISE', 'catalog': 'allwise_p3as_psd',
                                'position': target, 'SearchMethod': 'Cone', 'radius': 1200},
                  options={'removable': True, 'showUnits': False, 'showFilters': True})
    
    
# Add first chart - scatter (plot.ly direct plot)
# in cell 0, 0, 2, 2, 
viewer_id = 'newChartContainer'
r = fc.add_cell(0, 0, 2, 2,'xyPlots', viewer_id)

if r['success']:
    trace1 = {
        'tbl_id': 'wiseCatTbl',
        'x': "tables::w1mpro-w2mpro",
        'y': "tables::w2mpro-w3mpro",
        'mode': 'markers',
        'type': 'scatter', 
        'marker': {'size': 4}}
    trace_data=[trace1]
    
    layout_s = {'title': 'Color-Color', 
                'xaxis': {'title': 'w1mpro-w2mpro (mag)'}, 'yaxis': {'title': 'w2mpro-w3mpro (mag)'}}   
    fc.show_chart(group_id=viewer_id, layout=layout_s, data=trace_data )
    
# Add second chart - heatmap  (plot.ly direct plot) 
# in cell 2, 0, 2, 3
viewer_id = 'heatMapContainer'
r = fc.add_cell(2, 0, 2, 3,'xyPlots', viewer_id)

if r['success']:
    dataHM = [
                {
                    'type': 'fireflyHeatmap',
                    'name': 'w1 vs. w2',
                    'tbl_id': 'wiseCatTbl',
                    'x': "tables::w1mpro",
                    'y': "tables::w2mpro",
                    'colorscale': 'Blues'
                },
                {
                    'type': 'fireflyHeatmap',
                    'name': 'w1 vs. w3',
                    'tbl_id': 'wiseCatTbl',
                    'x': "tables::w1mpro",
                    'y': "tables::w3mpro",
                    'colorscale': 'Reds',
                    'reversescale': True
                },
                {
                    'type': 'fireflyHeatmap',
                    'name': 'w1 vs. w4',
                    'tbl_id': 'wiseCatTbl',
                    'x': "tables::w1mpro",
                    'y': "tables::w4mpro",
                    'colorscale': 'Greens'
                }]

    layout_hm = {'title': 'Magnitude-magnitude densities', 
                 'xaxis': {'title': 'w1 photometry (mag)'}, 'yaxis': {'title': ''},
                 'firefly': {'xaxis': {'min': 5, 'max': 20}, 'yaxis': {'min': 4, 'max': 18}}}  

    fc.show_chart(group_id=viewer_id, layout=layout_hm, data=dataHM)    
    
# Add third chart - histogram  (plot.ly direct plot) 
# in cell 2, 2, 2, 3
viewer_id =  'histContainer'
r = fc.add_cell(2, 2, 2, 3, 'xyPlots', viewer_id)

if r['success']:
    dataH = [
        {
            'type': 'fireflyHistogram',
            'name': 'w1mpro', 
            'marker': {'color': 'rgba(153, 51, 153, 0.8)'},
            'firefly': {
                'tbl_id': 'wiseCatTbl',
                'options': {
                    'algorithm': 'fixedSizeBins',
                    'fixedBinSizeSelection': 'numBins',
                    'numBins': 30,
                    'columnOrExpr': 'w1mpro'
                }
            },
        },
        {
            'type': 'fireflyHistogram',
            'name': 'w2mpro', 
            'marker': {'color': 'rgba(102, 153, 0, 0.7)'},
            'firefly': {
                'tbl_id': 'wiseCatTbl',
                'options': {
                    'algorithm': 'fixedSizeBins',
                    'fixedBinSizeSelection': 'numBins',
                    'numBins': 40,
                    'columnOrExpr': 'w2mpro'   
                }
            }
        }
    ]

    layout_hist = {'title': 'Photometry histogram',
                   'xaxis': {'title': 'photometry (mag)'}, 'yaxis': {'title': ''}}   
    result = fc.show_chart(group_id=viewer_id, layout=layout_hist, data=dataH )  
    
# Add fourth chart - 3D plot (plot.ly direct plot)
# in cell 2, 4, 2, 3, 
viewer_id =  '3dChartContainer'
r = fc.add_cell(2, 4, 2, 3, 'xyPlots', viewer_id)

if r['success']:
    data3d = [
        {
            'type': 'scatter3d',
            'name': 'w1-w2-w3',
            'tbl_id': 'wiseCatTbl',
            'x': "tables::w1mpro",
            'y': "tables::w2mpro",
            'z': "tables::w3mpro",
            'mode' : 'markers',
            'marker' : {
                'size': 3,
                'color': 'rgba(127, 127, 127, 1)',
                'line': {
                    'color': 'rgba(127, 127, 127, 0.5)',
                    'width': 1}
            },
            'hoverinfo': 'x+y+z'
        }]

    fsize = {'size': 11}
    layout3d = {
        'title': 'Photometry in band 1, 2, 3',
        'scene':{
            'xaxis': {
                'title': 'w1 (mag)',
                'titlefont': fsize
            },
            'yaxis': {
                'title': 'w2 (mag)',
                'titlefont': fsize
            },
            'zaxis': {
                'title': 'w3 (mag)',
                'titlefont': fsize
            }
        }
    }
    
    fc.show_chart(group_id=viewer_id, is_3d=True, layout=layout3d, data=data3d )        
    
# Add a three color fits
# in cell 0, 2, 2, 2
viewer_id = '3C'
r = fc.add_cell(0, 2, 2, 2, 'images', viewer_id)
rv = '92,-2,92,8,NaN,2,44,25,600,120'
if r['success']:
    threeC= [
         {
             'Type'      : 'SERVICE',
             'Service'   : 'WISE',
             'Title'     : '3 color',
             'SurveyKey'  : 'Atlas',
             'SurveyKeyBand': '1',
             'WorldPt'    : target,
             'RangeValues': rv,
             'SizeInDeg'  : '1'
         },
         {
             'Type'      : 'SERVICE',
             'Service'   : 'WISE',
             'Title'     : '3 color',
             'SurveyKey'  : 'Atlas',
             'SurveyKeyBand': '2',
             'WorldPt'    : target,
             'RangeValues': rv,
             'SizeInDeg'  : '1'
         },
         {
             'Type'      : 'SERVICE',
             'Service'   : 'WISE',
             'Title'     : '3 color',
             'SurveyKey'  : 'Atlas',
             'SurveyKeyBand': '3',
             'WorldPt'    : target,
             'RangeValues': rv,
             'SizeInDeg'  : '1'
         }]
        
    fc.show_fits_3color(threeC, viewer_id=viewer_id)